In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# silence warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [ ]:
device.reset()

In [4]:
! ../keras-retinanet/keras_retinanet/bin/train.py --weights ../vm_mount/mids-251-elephant-seal/Data/image-level-split/snapshots/resnet50_csv_11.h5 \
--batch-size 2 --steps 100 --epochs 10 \
--snapshot-path snapshots-no \
--random-transform \
--config config.ini \
csv image-level-split/train/annotations_final_no.csv classes-no.csv \
--val-annotations image-level-split/valid/annotations_final_no.csv

Using TensorFlow backend.
Creating model, this may take a second...
Model: "retinanet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9408        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu (Activation)         (None, None, None, 6 0           bn_conv1[0][0]                   
______________________

In [7]:
# convert the model weights to a usable format
#! ../keras-retinanet/keras_retinanet/bin/convert_model.py --config config.ini  snapshots-no/resnet50_csv_10.h5 harbor_seal_model_image_level_no_other.h5
# run the model on test set
! ../keras-retinanet/keras_retinanet/bin/evaluate.py csv image-level-split/test/annotations_final_no.csv classes-no.csv harbor_seal_model_image_level_no_other.h5


Using TensorFlow backend.
Loading model, this may take a second...
Running network: 100% (38 of 38) |#######| Elapsed Time: 0:02:43 Time:  0:02:43
Parsing annotations: 100% (38 of 38) |###| Elapsed Time: 0:00:00 Time:  0:00:00
291 instances of class adult with average precision: 0.9542
Inference time for 38 images: 4.2812
mAP using the weighted average of precisions among classes: 0.9542
mAP: 0.9542


In [8]:
%matplotlib inline

%reload_ext autoreload
%autoreload 2

import keras

from keras_retinanet.models import retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import json
from random import shuffle

import tensorflow as tf

def get_session():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.compat.v1.Session(config=config)

tf.compat.v1.keras.backend.set_session(get_session())

Using TensorFlow backend.


In [9]:
model_path = 'harbor_seal_model_image_level.h5'

print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')

#print(model.summary())

labels_to_names = {0: 'adult', 1:'other'}

harbor_seal_model_image_level.h5


In [ ]:
image_list_no_seals =[
    "test/12HS0261-22_png.rf.977d17cdf82eda5e77b0542303878b29.jpg",
    "test/12HS0263-107_png.rf.429e3e3d811bc51e773669ef5d91cd14.jpg",
    "test/12HS6096-49_png.rf.7262616772c824c8d2e43dbf3dbebb34.jpg",
    "test/12HS6096-92_png.rf.baa2bb57b553be53ec42637e134c23f9.jpg",
    "test/12HS6116-82_png.rf.d370c2abf6f8199b7ffd76bf74d11f19.jpg",
    "test/12HS6116-83_png.rf.b56184af20a62c6da6c7d4a71cadf900.jpg",
    "test/12HS6120-73_png.rf.5ebb1f12c0fe4f87c9ea7df215beda21.jpg",
    "test/12HS6153-12_png.rf.eedb5412e84f37af15a1fed4e5937cf6.jpg",
    "test/12HS8978-35_png.rf.c1d0d010eaa61a0662dda66c81b2e87b.jpg"
]
print(len(image_list_no_seals))

In [ ]:
visualize = True
min_score = 0.5

detections = {}

total_time = 0

for image_path in image_list_no_seals: 
    image = read_image_bgr(image_path)
    
    if visualize:
        draw = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
        
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start 
    
    boxes /= scale
    if any(score >= min_score for score in scores [0]):
        detections[image_path] = []
    
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < min_score:
            break
        
        b = box.astype(int)
        detections[image_path].append({"box" : b, "label" : label, "score" : score})
        
        if visualize: 
            color = label_color(label)
            
            draw_box(draw, b, color=color)
            
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)
            
    if any(score >= min_score for score in scores[0]):
        if visualize:
            print(image_path)
            plt.figure(figsize=(15, 15))
            plt.axis('off')
            plt.imshow(draw)
            plt.show()
            
print("Finished, time per image:", total_time/len(image_list_no_seals))

In [11]:
image_list_too_many_seals =[
"12HS0263-105_png.rf.c2bbaab8711057ff0dbf7fe1bab58ef1.jpg",
"12HS0263-106_png.rf.f93415a151eb218963aec00e6768a444.jpg",
"12HS0263-120_png.rf.b2827b935ba211dbdb59bfda487dfca9.jpg",
"12HS0263-121_png.rf.b3147f8f90eca243e871cadf8635eae6.jpg",
"12HS0263-162_png.rf.747983d3e4d6f328661738dfc6de2887.jpg",
"12HS0263-163_png.rf.45b67a237445956466317a213ff5a723.jpg",
"12HS0263-21_png.rf.4fa6477df74b0b554aae687cefc4a5f3.jpg" ,
"12HS0263-22_png.rf.a0fce6ed8c332e4e6280253ccbd04cd2.jpg" ,
"12HS0263-35_png.rf.ea947d5c9c3fa76eceaac4fb1156112a.jpg" ,
"12HS0263-36_png.rf.826fc11f84f59b19e43ee18237ecb913.jpg" ,
"12HS0263-48_png.rf.b0d18b77239c19786a4735b80fd4cc57.jpg" ,
"12HS0263-49_png.rf.b75816bf926c655d6514b56919729fc3.jpg" ,
"12HS0263-50_png.rf.a5f451b8b8313c00bcd13e81398bd16d.jpg" ,
"12HS0263-62_png.rf.69766779fae74eef2c1d1974dd0280cd.jpg" ,
"12HS0263-63_png.rf.731a72ccd42280be862049dda5df254e.jpg" ,
"12HS0263-64_png.rf.90157084cfcded827e0d635bdc1898a6.jpg" ,
"12HS0263-77_png.rf.9f2c2392446a1d6c883b520828177638.jpg" ,
"12HS0263-78_png.rf.9fb75e09133b0c6a42bafdfcd7f804f0.jpg" ,
"12HS0263-7_png.rf.91acea8d0c241a862c237f7e36e2c27a.jpg"  ,
"12HS0263-8_png.rf.0e9df59119ad0529445bb95ffa3f22f0.jpg"  ,
"12HS0263-90_png.rf.c029370784fa182a1dd47c909183bbda.jpg" ,
"12HS0263-91_png.rf.c770db10bd2405254374a4f1cc1ad4cb.jpg" ,
"12HS0263-92_png.rf.8ce0c5871433abe64bcef954c0637553.jpg" ,
"12HS6115-107_png.rf.35acaa2a38fcde204b24f93103d9742a.jpg",
"12HS6115-108_png.rf.4783abcc9bfbc57daa5c6988282d1c61.jpg",
"12HS6115-121_png.rf.6771a635badf08af530eb37df56f4c14.jpg",
"12HS6115-122_png.rf.418c1325ca7c3587a842104e51be0d47.jpg",
"12HS6115-160_png.rf.dcaeb8934f171750bb28ec1666700805.jpg",
"12HS6115-164_png.rf.9234e29dae07aa7800da1aefdb4bde61.jpg",
"12HS6115-78_png.rf.2d6a6e2873b450fee0df2a3f67a1df88.jpg", 
"12HS6115-79_png.rf.3784417e5503562a9ac7c882a12cf02b.jpg",
"12HS6115-8_png.rf.ca2ac3f7501b2cde33cc00d31d85a920.jpg",  
"12HS6115-93_png.rf.3d2715f106b081149245e5f6d5b959a0.jpg", 
"12HS6115-94_png.rf.1be9d0117c8108640120ed341e18da57.jpg", 
]
print(len(image_list_too_many_seals))

34


In [12]:
detections = {}

total_time = 0
visualize = True

for image_path in image_list_too_many_seals: 
    full_image_path = "image-level-split/test/" + image_path
    image = read_image_bgr(full_image_path)
    
    if visualize:
        draw = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
        
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start 
    
    boxes /= scale
    if any(score >= min_score for score in scores [0]):
        detections[image_path] = []
    
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < min_score:
            break
        
        b = box.astype(int)
        detections[image_path].append({"box" : b, "label" : label, "score" : score})
        
        if visualize: 
            color = label_color(label)
            
            draw_box(draw, b, color=color)
            
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)
            
    if any(score >= min_score for score in scores[0]):
        if visualize:
            print(image_path)
            plt.figure(figsize=(15, 15))
            plt.axis('off')
            plt.imshow(draw)
            plt.show()
            
print("Finished, time per image:", total_time/len(image_list_too_many_seals))

NameError: name 'min_score' is not defined

In [14]:
image_dir = "image-level-split/test/"

image_list = []
for root, dirs, files in os.walk(image_dir):
    for filename in files:
        if filename.lower().endswith(('.jpg')) and "checkpoint" not in filename:
            image_list.append(image_dir + filename)
print(len(image_list))

42


In [15]:
min_score = 0.5

detections = {}

total_time = 0

for image_path in image_list:
    
    image = read_image_bgr(image_path)
          
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    total_time += time.time() - start
    
    boxes /= scale  
        
    if any(score >= min_score for score in scores[0]):
        detections[image_path] = []
        
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < min_score:
            break
        
        b = box.astype(int)    
        detections[image_path].append({"box" : b, "label" : label, "score" : score})
        
            
print("Finished, time per image:", total_time/len(image_list))

Finished, time per image: 0.7857930944079444


In [16]:
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [18]:
with open('image-level-split/test_detections-no-other.json', 'w') as fp:
    json.dump(detections, fp, cls=MyEncoder)